In [1]:
# Importing and initializing findspark.
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/25 22:19:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep = ",", header = True, inferSchema = True)

In [4]:
# Showing top five rows
home_sales_df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [5]:
# Showing DataFrame schema
home_sales_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



24/05/25 22:20:01 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [6]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView("home_sales")

In [7]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

# Writing and executing the SQL query
result_3 = spark.sql("""
    SELECT 
        YEAR(date) AS year, 
        ROUND(AVG(price), 2) AS avg_price 
    FROM 
        home_sales 
    WHERE 
        bedrooms = 4 
    GROUP BY 
        YEAR(date)
    ORDER BY 
        year
""")

# Showing the result
result_3.show()

+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [8]:
# 4. What is the average price of a home for each year the home was built, that have 3 bedrooms and 3 bathrooms,
# rounded to two decimal places?

# Writing and executing the SQL query
result_4 = spark.sql("""
    SELECT 
        date_built AS year_built, 
        ROUND(AVG(price), 2) AS avg_price 
    FROM 
        home_sales 
    WHERE 
        bedrooms = 3 AND 
        bathrooms = 3 
    GROUP BY 
        date_built
    ORDER BY 
        date_built
""")

# Showing the result
result_4.show()

+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [9]:
# 5. What is the average price of a home for each year the home was built, that have 3 bedrooms, 3 bathrooms,
# with two floors, and are greater than or equal to 2,000 square feet, rounded to two decimal places?

# Writing and executing the SQL query
result_5 = spark.sql("""
    SELECT 
        date_built AS year_built, 
        ROUND(AVG(price), 2) AS avg_price 
    FROM 
        home_sales 
    WHERE 
        bedrooms = 3 AND 
        bathrooms = 3 AND 
        floors = 2 AND 
        sqft_living >= 2000 
    GROUP BY 
        date_built
    ORDER BY 
        date_built
""")

# Showing the result
result_5.show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [10]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

# Writing and executing the SQL query
result_6 = spark.sql("""
    SELECT 
        view, 
        ROUND(AVG(price), 2) AS avg_price 
    FROM 
        home_sales 
    GROUP BY 
        view
    HAVING 
        AVG(price) >= 350000
    ORDER BY 
        view DESC
""")

# Showing the result
result_6.show()

# Printing the run time for the query
print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.19150519371032715 seconds ---


In [11]:
# Checking the number of rows in the result
result_6.count()

50

In [12]:
# Showing all rows in the result
result_6.show(50)

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
|  80| 991767.38|
|  79|1009565.08|
|  78|1080649.37|
|  77|1076205.56|
|  76|1058802.78|
|  75|1114042.94|
|  74|  745077.0|
|  73| 752861.18|
|  72| 780914.67|
|  71|  775651.1|
|  70| 695865.58|
|  69| 750537.94|
|  68| 716785.44|
|  67| 737970.96|
|  66|  712475.0|
|  65| 736679.93|
|  64| 767036.67|
|  63| 711614.55|
|  62| 759150.14|
|  61| 746877.59|
|  60| 754939.65|
|  59|  791453.0|
|  58| 759764.65|
|  57|  734340.5|
|  56|  718176.4|
|  55| 771153.32|
|  54| 798684.82|
|  53|  755214.8|
|  52| 733780.26|
|  51| 788128.21|
+----+----------+



In [15]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [16]:
# 8. Check if the table is cached.
spark.catalog.isCached("home_sales")

True

In [17]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

# Writing and executing the SQL query
result_9 = spark.sql("""
    SELECT 
        view, 
        ROUND(AVG(price), 2) AS avg_price 
    FROM 
        home_sales 
    GROUP BY 
        view
    HAVING 
        AVG(price) >= 350000
    ORDER BY 
        view DESC
""")

# Showing the result
result_9.show()

# Printing the run time for the query
print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.12412118911743164 seconds ---


In [18]:
# Checking the number of rows in the result
result_9.count()

50

In [19]:
# Showing all rows in the result
result_9.show(50)

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
|  80| 991767.38|
|  79|1009565.08|
|  78|1080649.37|
|  77|1076205.56|
|  76|1058802.78|
|  75|1114042.94|
|  74|  745077.0|
|  73| 752861.18|
|  72| 780914.67|
|  71|  775651.1|
|  70| 695865.58|
|  69| 750537.94|
|  68| 716785.44|
|  67| 737970.96|
|  66|  712475.0|
|  65| 736679.93|
|  64| 767036.67|
|  63| 711614.55|
|  62| 759150.14|
|  61| 746877.59|
|  60| 754939.65|
|  59|  791453.0|
|  58| 759764.65|
|  57|  734340.5|
|  56|  718176.4|
|  55| 771153.32|
|  54| 798684.82|
|  53|  755214.8|
|  52| 733780.26|
|  51| 788128.21|
+----+----------+



In [20]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data.
home_sales_df.write.partitionBy("date_built").parquet("parquet_home_sales", mode = "overwrite")

In [21]:
# 11. Read the formatted parquet data.
p_home_sales_df = spark.read.parquet("parquet_home_sales")

In [23]:
# 12. Create a temporary table for the parquet data.
p_home_sales_df.createOrReplaceTempView("p_home_sales")

In [24]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

# Writing and executing the SQL query
result_13 = spark.sql("""
    SELECT 
        view, 
        ROUND(AVG(price), 2) AS avg_price 
    FROM 
        p_home_sales 
    GROUP BY 
        view
    HAVING 
        AVG(price) >= 350000
    ORDER BY 
        view DESC
""")

# Showing the result
result_13.show()

# Printing the run time for the query
print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.2510409355163574 seconds ---


In [25]:
# Checking the number of rows in the result
result_13.count()

50

In [26]:
# Showing all rows in the result
result_13.show(50)

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
|  80| 991767.38|
|  79|1009565.08|
|  78|1080649.37|
|  77|1076205.56|
|  76|1058802.78|
|  75|1114042.94|
|  74|  745077.0|
|  73| 752861.18|
|  72| 780914.67|
|  71|  775651.1|
|  70| 695865.58|
|  69| 750537.94|
|  68| 716785.44|
|  67| 737970.96|
|  66|  712475.0|
|  65| 736679.93|
|  64| 767036.67|
|  63| 711614.55|
|  62| 759150.14|
|  61| 746877.59|
|  60| 754939.65|
|  59|  791453.0|
|  58| 759764.65|
|  57|  734340.5|
|  56|  718176.4|
|  55| 771153.32|
|  54| 798684.82|
|  53|  755214.8|
|  52| 733780.26|
|  51| 788128.21|
+----+----------+



In [27]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [28]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")

False

In [29]:
# Stopping the Spark session
spark.stop()